In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import sys
from datetime import datetime
from datetime import timedelta
import xlsxwriter

# Data Processing helpers

In [2]:
data_path = r"C:\Users\User\Desktop\Math402w\Math402W Proj\907\afternoon" 
# put r before your normal string it converts normal string to raw string:

#processes the time matrix
time_matrix = pd.read_csv(data_path+'/time_matrix.csv', index_col = 'Unnamed: 0')
processed_time_matrix = time_matrix.values.tolist()

In [3]:
time_slot_data = pd.read_csv(data_path+'/time_slot.csv')
#pd.read_csv(data_path+'/903_time_slot_AM.csv', index_col = 'Unnamed: 0')
#creates a tuple of time data
processed_time_windows = time_slot_data[["start_time_scaled","end_time_scaled"]].apply(tuple, axis = 1).to_list()

In [4]:
##
time_slot_data_forcsv = pd.read_csv(data_path+'/time_slot.csv', index_col=[0])
time_slot_data_forcsv=time_slot_data_forcsv.reset_index()
time_slot_data_forcsv.rename(columns={'dummy_add':'address'}, inplace=True )
time_slot_data_forcsv=time_slot_data_forcsv.replace('\+',' ',regex=True)
time_slot_data_forcsv.drop(['slots', 'start_time_scaled','end_time_scaled'], axis = 1,inplace = True)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('907_afternoon_10,000_schedule.xlsx', engine='xlsxwriter')

# Config

In [5]:
#helper functions
def addServiceTime(time_matrix, service_time):
    for from_idx in range(1,len(time_matrix)):
        for to_idx in range(len(time_matrix[0])):
            if from_idx != to_idx:
                time_matrix[from_idx][to_idx] = service_time + time_matrix[from_idx][to_idx]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = processed_time_matrix
    data['time_windows'] = processed_time_windows
    data['num_vehicles'] = 1 # to be updated
    data['num_locations'] = len(data['time_windows']) - 1
    data['depot'] = 0 # depot location based on item 
    
    #driver breaks
    data['driver_break_duration'] = 30 # to be updated
    data['driver_break_start'] = 195  # 3 hours 15mins past 6:45am(4 hours past 6am)  - to be updated
    data['driver_break_end'] = 255     # 60mins time windows, to be updated 
    data['breaks'] = [data['driver_break_duration'] for _ in range(data['num_vehicles'])]
    
    #service times
    data['service_time'] = 12 # to be updated
    data['maximum_wait_at_location'] = 10000
    addServiceTime(data['time_matrix'], data['service_time'])
    
    
    #vehicle capacities
    data['demands'] = [0] + [1 for _ in range(data['num_locations'])]
    data['max_vehicle_capacities'] = 21
    load_balancing = True; # to be updated
    limit_vehicle_capacity = 0.8  #only applied if load_balancing = true
    
    if load_balancing == True:
        data['vehicle_capacities'] = [data['max_vehicle_capacities']* limit_vehicle_capacity for _ in range(data['num_vehicles'])]
    else:
        data['vehicle_capacities'] = [data['max_vehicle_capacities'] for _ in range(data['num_vehicles'])]
    
    #solver config
    data['solution_limit'] = 10000 #40000 #upper bound of # of solutions to calculate
    data['drop_penalty'] = 99999
    
    #set time boundaries

    return data


def print_solution(data, manager, routing, solution, dropped_address):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    #prints breaks
    print('Breaks:')
    intervals = solution.IntervalVarContainer()
    for i in range(intervals.Size()):
        brk = intervals.Element(i)
        if brk.PerformedValue() == 1:
            print('{}: Start({}) Duration({})'.format(
                brk.Var().Name(),
                brk.StartValue(),
                brk.DurationValue()))
            ##
            break_value = brk.StartValue()
            
        else:
            print('{}: Unperformed'.format(brk.Var().Name()))
    print()
    
    #Prints dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
            a = time_slot_data_forcsv[time_slot_data_forcsv['index']==node]
            dropped_address = dropped_address.append(a,ignore_index= True)
    print(dropped_nodes, '\n')
    
    
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    
    ##
    current_time_str = '17/03/2022 14:45'   #to be update(6:45 for morning shift or 14:45 for afternoon shift)
    date_format_str = '%d/%m/%Y %H:%M'
    # create datetime object from timestamp string
    current_time = datetime.strptime(current_time_str, date_format_str)
    driver_shift_start_time = current_time - timedelta(minutes = 45)
    start_location = ['0',"Add_0", driver_shift_start_time, current_time]
    sheet_num=1 #use to name the tabs in excel 
    break_start_time = current_time + pd.DateOffset(minutes=break_value)
    break_end_time = break_start_time + pd.DateOffset(minutes=30)
    start_location = ['0',"Add_0", current_time - timedelta(minutes = 45), current_time]
    
    for vehicle_id in range(data['num_vehicles']):
        i = 1  
        final=pd.DataFrame(columns=['index','address','start_time', 'end_time'])     
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)      
        isFirstNode = True
        
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            
            if isFirstNode:          
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var))
                final.loc[0] = start_location 
                            
                index = solution.Value(routing.NextVar(index))
                isFirstNode = False

                n=solution.Min(time_var)
                a = time_slot_data_forcsv[time_slot_data_forcsv['index']==index]
                final=final.append(a,ignore_index= True)
                final.loc[i,'start_time'] = current_time  + timedelta(minutes = n) 
                end_time = final.loc[i,'start_time']   + timedelta(minutes=data['service_time'])
                final.loc[i,'end_time'] = end_time
                check_value_last_end = solution.Min(time_var) # add here so we can check at the first iteration
                i+=1
                
            else:
                plan_output += '{0} Time({1},{2}) -> '.format(
                    manager.IndexToNode(index), solution.Min(time_var),
                    solution.Max(time_var) + data["service_time"])
                index = solution.Value(routing.NextVar(index))
                 ##
                check_value = solution.Min(time_var)
                
                if (check_value_last_end <= break_value) & (check_value > break_value): # if break happens after the last order and before next 
                    final.loc[i] = ["break_time", "break_time", break_start_time, break_end_time ]
                    n=solution.Min(time_var) + 30
                    i=i+1
                else:
                    n=solution.Min(time_var)   
                    
                a = time_slot_data_forcsv[time_slot_data_forcsv['index']==index]
                final=final.append(a,ignore_index= True)
                final.loc[i,'start_time'] = current_time  + timedelta(minutes = n) 
                end_time = final.loc[i,'start_time']   + timedelta(minutes=data['service_time'])
                final.loc[i,'end_time'] = end_time
                check_value_last_end = solution.Min(time_var) 
                i+=1

        final.loc[final.index[-1],'index'] = 0
        final.loc[final.index[-1],'address'] = "Add_0"
        final.loc[final.index[-1],"end_time"] = "shift finished"       
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))    
       
        print(plan_output)
        total_time += solution.Min(time_var)
        ## import multiple dataframes into different worksheets
        final.to_excel(writer, sheet_name="Driver" + str(sheet_num),index=False)
        sheet_num+=1
        
    writer.save()    
    dropped_address.to_csv('907_afternoon_10000_dropped_address.csv', index=False)
  
    print('Total time of all routes: {}min'.format(total_time))
    




# TSP Solver

In [6]:
"""Solve the VRP with time windows."""

def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                       data['num_vehicles'], data['depot'])

    #used for tracing/debugging
    # routing_parameters = pywrapcp.DefaultRoutingModelParameters()
    # routing_parameters.solver_parameters.trace_propagation = True
    # routing_parameters.solver_parameters.trace_search = True

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager) #, routing_parameters)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Adds Capacity Constraints
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        10000000,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Allow to drop nodes.
    penalty = data['drop_penalty']
    for node in range(1, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        data['maximum_wait_at_location'],  # allow waiting time
        data['maximum_wait_at_location'],  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)


    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
        
    
    # Add driver break constraints
    node_visit_transit = {}
    for n in range(routing.Size()):
        if n >= len(data['time_windows']):
            node_visit_transit[n] = 0
        else:
            node_visit_transit[n] = int(data['demands'][n] * data['service_time'])

    break_intervals = {}
    for v in range(data['num_vehicles']):
        vehicle_break = data['breaks'][v]
        break_intervals[v] = [
            routing.solver().FixedDurationIntervalVar(
                data['driver_break_start'], 
                data['driver_break_end'],
                data['driver_break_duration'],
                False,
                'Break for vehicle {}'.format(v))
        ]
        time_dimension.SetBreakIntervalsOfVehicle(
            break_intervals[v], v, node_visit_transit)

    

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
       routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.log_search = True
    search_parameters.solution_limit = data['solution_limit']


    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    dropped_address = pd.DataFrame(columns=['index','address']) # create a df to hold the dropped address
    if solution:
        print_solution(data, manager, routing, solution, dropped_address)
    else:
        print("no solutions found")
        
if __name__ == '__main__':
    main()
     #main(sys.argv[1])

Objective: 196
Breaks:
Break for vehicle 0: Start(242) Duration(30)

Dropped nodes: 

Route for vehicle 0:
0 Time(93,93) -> 8 Time(103,115) -> 12 Time(120,132) -> 9 Time(135,147) -> 1 Time(147,159) -> 6 Time(160,172) -> 2 Time(173,185) -> 11 Time(187,199) -> 3 Time(204,216) -> 4 Time(222,234) -> 7 Time(235,247) -> 5 Time(279,309) -> 10 Time(315,327) -> 0 Time(337,337)
Time of the route: 337min

Total time of all routes: 337min
